#### CRIAR DATASET

In [2]:
from PIL import Image
import os

input_dir = "../data/post-processed"
output_dir = "../data/normalized"
target_size = (224, 224)

# Dimensões do bloco
block_size = 24

# Até o bloco 50: 6 linhas completas + 5 blocos da 7ª linha
# Portanto, cortaremos até y = 144
crop_height = 144  # 6 blocos * 24 px

os.makedirs(output_dir, exist_ok=True)

for person in os.listdir(input_dir):
    person_path = os.path.join(input_dir, person)
    if not os.path.isdir(person_path):
        continue

    output_person_path = os.path.join(output_dir, person)
    os.makedirs(output_person_path, exist_ok=True)

    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)

        try:
            with Image.open(img_path) as img:
                img = img.convert("RGB")
                img_resized = img.resize(target_size)

                # Corta somente a parte até o bloco 50
                img_cropped = img_resized.crop((0, 0, 224, crop_height))  # (left, top, right, bottom)
                
                img_padded = Image.new("RGB", (224, 224), (0, 0, 0))
                img_padded.paste(img_cropped, (0, 0))

                save_path = os.path.join(output_person_path, img_name)
                img_padded.save(save_path)

        except Exception as e:
            print(f"Erro ao processar {img_path}: {e}")

print("Imagens redimensionadas para 224x224 e cortadas até o bloco 50")


Imagens redimensionadas para 224x224 e cortadas até o bloco 50


#### VISUALIZAR IMAGEM COM BLOCOS

In [3]:
import cv2
import numpy as np

# Carrega a imagem
img = cv2.imread("C:/Users/romario.santos/Documents/desafio/data/marcelinho_no_db (1).jpg")
img = cv2.resize(img, (224, 224))  # Garante tamanho certo
save = "data/"

# Clona a imagem pra desenhar as divisões
img_blocks = img.copy()

# Define tamanho do bloco
block_size = 24
max_blocks = 50
count = 0

img_copy = img.copy()

# Itera sobre os blocos linha a linha
for y in range(0, 224, block_size):
    for x in range(0, 224, block_size):
        if count >= max_blocks:
            break

        # Desenha o retângulo do bloco
        cv2.rectangle(img_copy, (x, y), (x + block_size, y + block_size), (0, 255, 0), 1)
        count += 1
    if count >= max_blocks:
        break


cv2.imshow("Blocos 1 a 50", img_copy)
cv2.waitKey(0)

-1

#### PRE PROCESSAMENTO

In [ ]:
import cv2

img = cv2.imread("C:/Users/romario.santos/Downloads/Captura de tela 2025-04-09 095226.png")
img = cv2.resize(img, (224, 224))  # Garante tamanho exato

# Define região até o bloco 50
# Linhas 0 a 5: 6 linhas completas → 6 * 24 = 144 pixels
# Colunas 0 a 8: imagem toda (ou pode limitar para metade)
corte = img[0:144, 0:224]  # (y1:y2, x1:x2)

cv2.imshow("Região Olhos + Testa (blocos 0-50)", corte)
cv2.imwrite("macron_mascara.jpg", corte )
cv2.waitKey(0)
cv2.destroyAllWindows()


#### MODELOS

In [6]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Carrega o modelo VGG-16 pré-treinado
vgg16 = models.vgg16(pretrained=True)

# Queremos até a última conv, ou seja: só as features
feature_extractor = vgg16.features.eval()  # 13 conv layers + MaxPool

# Verifica a estrutura
print(feature_extractor)

c:\Users\romario.santos\AppData\Local\anaconda3\envs\Mobia\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\romario.santos\AppData\Local\anaconda3\envs\Mobia\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [7]:
# Carrega AlexNet pré-treinado
alexnet = models.alexnet(pretrained=True)

# As camadas convolucionais estão no .features
# Queremos até a camada 12 (conv5 + ReLU + MaxPool)
feature_extractor_alex = torch.nn.Sequential(*list(alexnet.features.children())[:13])  # Até conv5

feature_extractor_alex.eval()

c:\Users\romario.santos\AppData\Local\anaconda3\envs\Mobia\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [8]:

import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Carrega modelo ResNet-50 pré-treinado
resnet = models.resnet50(pretrained=True)
resnet.eval()


c:\Users\romario.santos\AppData\Local\anaconda3\envs\Mobia\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
# Remove avgpool e fc: queremos a saída bruta de conv5_x
# Isso é feito pegando tudo até o layer `layer4` (conv5_x é a saída de layer4)
from torch import nn

feature_extractor_resnet = nn.Sequential(
    resnet.conv1,
    resnet.bn1,
    resnet.relu,
    resnet.maxpool,
    resnet.layer1,  # conv2_x
    resnet.layer2,  # conv3_x
    resnet.layer3,  # conv4_x
    resnet.layer4   # ✅ conv5_x
).eval()


In [10]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

class MultiFeatureExtractor(nn.Module):
    def __init__(self):
        super(MultiFeatureExtractor, self).__init__()

        # VGG-16 até a última conv
        vgg = models.vgg16(pretrained=True)
        self.vgg_feat = nn.Sequential(*list(vgg.features.children())).eval()

        # AlexNet até conv5
        alex = models.alexnet(pretrained=True)
        self.alex_feat = nn.Sequential(*list(alex.features.children())[:13]).eval()

        # ResNet-50 até conv5_x (layer4)
        resnet = models.resnet50(pretrained=True)
        self.resnet_feat = nn.Sequential(
            resnet.conv1,
            resnet.bn1,
            resnet.relu,
            resnet.maxpool,
            resnet.layer1,
            resnet.layer2,
            resnet.layer3,
            resnet.layer4
        ).eval()

        # Global Average Pooling para todos
        self.gap = nn.AdaptiveAvgPool2d((1, 1))  # Transforma [C, H, W] → [C, 1, 1]

    def forward(self, x):
        # Passa pela VGG
        vgg_out = self.vgg_feat(x)         # [1, 512, 7, 7]
        vgg_vec = self.gap(vgg_out).squeeze()  # [512]

        # Passa pela AlexNet
        alex_out = self.alex_feat(x)       # [1, 256, 6, 6]
        alex_vec = self.gap(alex_out).squeeze()  # [256]

        # Passa pela ResNet-50
        resnet_out = self.resnet_feat(x)   # [1, 2048, 7, 7]
        resnet_vec = self.gap(resnet_out).squeeze()  # [2048]

        # Concatena tudo
        fused = torch.cat([vgg_vec, alex_vec, resnet_vec], dim=0)  # [2816]

        return fused  # ou: return vgg_vec, alex_vec, resnet_vec



In [11]:
# Instancia o modelo
multi_model = MultiFeatureExtractor()

# Transforma a imagem
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


#### CRIAR BANCO DE VETORES

In [12]:
# Inicializa modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiFeatureExtractor().to(device).eval()

# Transformações padrão
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
import os
import torch
from PIL import Image
import json
import numpy as np

# Caminhos
input_dir = "data/normalized"
output_file = "banco_de_embeddings.json"

# Estrutura do banco: chave = nome da pessoa, valor = lista de embeddings
banco_temp = {}

# Percorre o dataset
for person in os.listdir(input_dir):
    person_path = os.path.join(input_dir, person)
    if not os.path.isdir(person_path):
        continue

    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)
        try:
            with Image.open(img_path) as img:
                img = img.convert("RGB")
                # Se suas imagens já estão normalizadas, pode usar transform direto
                img_tensor = transform(img).unsqueeze(0).to(device)
            
            with torch.no_grad():
                embedding = model(img_tensor).squeeze().cpu().numpy()
            
            # Armazena a média em banco_temp: para cada pessoa, acumula os embeddings
            banco_temp.setdefault(person, []).append(embedding.tolist())

        except Exception as e:
            print(f"Erro ao processar {img_path}: {e}")

# Agora, para cada pessoa, calcule a média dos embeddings
banco = {}
for person, emb_list in banco_temp.items():
    # Converte a lista de embeddings para um tensor para facilitar a média
    emb_array = torch.tensor(emb_list, dtype=torch.float32)
    media_embedding = emb_array.mean(dim=0)
    banco[person] = {
        "id": person,
        "num_images": len(emb_list),
        "embedding_med": media_embedding.tolist()
    }

# Salva o banco como JSON
with open(output_file, "w") as f:
    json.dump(banco, f)

print(f"✅ Banco de embeddings (média) salvo em {output_file}")


✅ Banco de embeddings (média) salvo em banco_de_embeddings.json


In [ ]:
print("Entradas no banco de dados:", list(banco.keys()))


NameError: name 'banco' is not defined

#### ADICIONAR NOVA PESSOA

In [34]:
# 0. Definições prévias necessárias
target_size = (224, 224)  # Exemplo
crop_height = 144  # Exemplo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 1. Entrada: imagem nova + ID da pessoa
img_path = "marcelinho_no_db (1).jpg"
pessoa_id = "marcelo"

# 2. Processamento de imagem
img = Image.open(img_path).convert("RGB")
img_resized = img.resize(target_size)
img_cropped = img_resized.crop((0, 0, 224, crop_height))

img_padded = Image.new("RGB", (224, 224), (0, 0, 0))
img_padded.paste(img_cropped, (0, 0))
img_padded.show()
# 3. Extração de embedding
img_tensor = transform(img_padded).unsqueeze(0).to(device)

with torch.no_grad():
    embedding = model(img_tensor).squeeze().cpu().numpy().tolist()  # Já é lista

# 4. Atualização do banco (versão eficiente)
# Atualiza banco temporário
if pessoa_id not in banco_temp:
    banco_temp[pessoa_id] = []
banco_temp[pessoa_id].append(embedding)  # Sem .tolist() extra

# Atualiza apenas a pessoa atual no banco principal
emb_array = torch.tensor(banco_temp[pessoa_id], dtype=torch.float32)
media_embedding = emb_array.mean(dim=0).tolist()

banco[pessoa_id] = {
    "id": pessoa_id,
    "num_images": len(banco_temp[pessoa_id]),
    "embedding_med": media_embedding
}

print(f"✅ Pessoa {pessoa_id} atualizada. Imagens: {len(banco_temp[pessoa_id])}")


✅ Pessoa marcelo atualizada. Imagens: 1


In [35]:
print("Entradas no banco de dados:", list(banco.keys()))


Entradas no banco de dados: ['Aaron_Eckhart', 'Aaron_Guiel', 'Aaron_Peirsol', 'Aaron_Sorkin', 'Aaron_Tippin', 'Abbas_Kiarostami', 'Abdel_Aziz_Al-Hakim', 'Abdel_Nasser_Assidi', 'Abdoulaye_Wade', 'Abdullah', 'Abdullah_Gul', 'Abel_Pacheco', 'Abid_Hamid_Mahmud_Al-Tikriti', 'Abraham_Foxman', 'Adam_Ant', 'Adam_Freier', 'Adam_Kennedy', 'Adam_Mair', 'Adam_Rich', 'Adam_Sandler', 'Adolfo_Aguilar_Zinser', 'Adolfo_Rodriguez_Saa', 'Adoor_Gopalakarishnan', 'Adrian_Annus', 'Adrian_Fernandez', 'Adrian_Murrell', 'Adrian_Nastase', 'Adrien_Brody', 'Afton_Smith', 'Agnelo_Queiroz', 'Ahmed_Ahmed', 'Ahmed_Chalabi', 'Ahmed_Qureia', 'Ahmet_Demir', 'Ahmet_Necdet_Sezer', 'Aicha_El_Ouafi', 'Aileen_Riggin_Soule', 'Aishwarya_Rai', 'Aitor_Gonzalez', 'Ai_Sugiyama', 'Ajit_Agarkar', 'AJ_Lamas', 'Akbar_Hashemi_Rafsanjani', 'Akhmed_Zakayev', 'Alain_Ducasse', 'Alanis_Morissette', 'Alanna_Ubach', 'Alan_Ball', 'Alan_Dershowitz', 'Alan_Greenspan', 'Alan_Mulally', 'Alan_Trammell', 'Alan_Zemaitis', 'Alastair_Campbell', 'Albert

#### VERIFICAR PESSOA NO BANCO

In [36]:
def reconhecer(img_path, banco, model, transform, target_size=(224,224), crop_height=144, threshold=0.7):
    """
    Função para reconhecer uma pessoa baseada no embedding médio armazenado no banco.
    - img_path: caminho para a imagem de teste.
    - banco: dicionário onde cada pessoa tem um embedding médio sob a chave "embedding_med".
    - model: modelo que extrai embeddings.
    - transform: transformações (inclui normalização) para a imagem.
    - target_size: tamanho para resize (default (224,224)).
    - crop_height: altura do crop da parte superior (ex.: 144 para 6 blocos de 24px).
    - threshold: valor de similaridade para considerar um match.
    """
    from torch.nn.functional import cosine_similarity
    from PIL import Image
    
    # Carrega a imagem com PIL e redimensiona
    img = Image.open(img_path).convert("RGB")
    img_resized = img.resize(target_size)
    
    # Corta a parte superior (por exemplo, testa/olhos)
    img_cropped = img_resized.crop((0, 0, target_size[0], crop_height))
    
    # Cria imagem de tamanho target com padding na parte inferior
    img_padded = Image.new("RGB", target_size, (0, 0, 0))
    img_padded.paste(img_cropped, (0, 0))
    img_padded.show()
    
    # Aplica transform para converter para tensor e normalizar
    img_tensor = transform(img_padded).unsqueeze(0).to(next(model.parameters()).device)
    
    # Extrai o embedding da imagem de teste
    with torch.no_grad():
        embedding_novo = model(img_tensor).squeeze().cpu()
    
    melhor_score = -1
    pessoa_mais_provavel = "desconhecido"
    
    for key, entry in banco.items():
        try:
            # Supondo que o banco armazena o embedding médio na chave "embedding_med"
            emb_vec = torch.tensor(entry["embedding_med"], dtype=torch.float32)
            # Opcional: normalizar ambos embeddings para que a comparação seja robusta
            embedding_novo_norm = torch.nn.functional.normalize(embedding_novo, p=2, dim=0)
            emb_vec_norm = torch.nn.functional.normalize(emb_vec, p=2, dim=0)
            
            score = cosine_similarity(embedding_novo_norm.unsqueeze(0), emb_vec_norm.unsqueeze(0), dim=1).item()
            
            if score > melhor_score:
                melhor_score = score
                pessoa_mais_provavel = entry["id"]
                
        except Exception as e:
            print(f"Erro ao comparar com {key}: {e}")
    
    if melhor_score >= threshold:
        return pessoa_mais_provavel, melhor_score
    else:
        return "desconhecido", melhor_score


In [37]:
img_path = "marcelinho_na_inferencia (1).jpg"
pessoa, score = reconhecer(img_path, banco, model, transform)

print(f"🔎 Resultado: {pessoa} (similaridade: {score:.3f})")


🔎 Resultado: marcelo (similaridade: 0.878)
